# Pokemon scraper

Gebruikte libraries

In [3]:
import requests
import math
import time
import pandas as pd
from bs4 import BeautifulSoup
import mysql.connector

### PokemonDB name scraper 

In [4]:
def scrapeDBName(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)

    pokemon_name = soup.find("h1").get_text()
    #Ethiek
    time.sleep(2)  
    return {"PokemonName": pokemon_name}

### Pokemon Type function

In [5]:
def scrapeDBType(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    #POKEMON TYPE
    row_counter = 0

    #er zijn meerdere tables met deze class, maar omdat de data die we nodig hebben enkel in de 1ste is gebruiken we gewoon soup.find()
    pokemon_type_table = soup.find("table",{"class": "vitals-table"})

    for table_row in pokemon_type_table.find_all("tr"):
        row_counter += 1
        
        if row_counter == 2:
            pokemon_type = table_row.find("td").get_text().strip()

            if(" " in pokemon_type != True):                               
                type_list= []
                type_list = pokemon_type.split()               
                Type1 = type_list[0]
                Type2 = type_list[1]
                return {"Type1": Type1, "Type2" : Type2}
 
            else:            
                return {"Type1": pokemon_type, "Type2" : "None"} 
    
    time.sleep(1)

In [6]:
scrapeDBType("rayquaza")

{'Type1': 'Dragon', 'Type2': 'Flying'}

### Pokemon stats function

In [7]:
def scrapeDBStats(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    
    #POKEMON STATS
    table_counter = 0
    row_counter = 0
    
    pokemonName_stats = []

    #er zijn meerdere tables met dezelfde class en geen unieke ID dus gaan we alle tables met deze class moeten scrapen
    all_stat_tables = soup.find_all("table",{"class": "vitals-table"})
    #loop om juiste table te vinden
    for stat_table in all_stat_tables:
        table_counter += 1
        
        #we  willen enkel de 4de table scrapen
        if table_counter == 4:
            
            #alle rijen in table loopen en table dimensions in rij vinden
            for table_row in stat_table.find_all("tr"):
                #print("row found")
                row_counter += 1
                #row_counter is nodig om "total" niet mee te nemen
                if row_counter <= 6:
                    dimensions = table_row.find_all("td")
                    header = table_row.find("th")
                    print(header.get_text() + ": " + dimensions[0].get_text())    
    #Ethiek
    time.sleep(1)  

### Pokemon Moves function

In [8]:
def scrapeDBMoves(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    #POKEMON MOVES
    #LEVEL MOVES
    ms_moves_list = []
    ms_moveType_list = []
    ms_moves_dict = {}

    levelup_moves_table = soup.find("table",{"class" : "data-table"})
    body_levelup_moves = levelup_moves_table.find("tbody")

    for move_row in body_levelup_moves.find_all("td",{"class":"cell-name"}):
        ms_moves_list.append(move_row.get_text())
    
    #move  type
    for move_row in body_levelup_moves.find_all("td",{"class":"cell-icon"}):
        ms_moveType_list.append((move_row.get_text()))

        #EGG MOVES
    table_counter = 0    
    tables = soup.find_all("table",{"class" : "data-table"})

    for table in tables:
        table_counter += 1
        
        if table_counter == 2:
            body_egg_moves = table.find("tbody")
            for move_row in body_egg_moves.find_all("td",{"class":"cell-name"}):
                ms_moves_list.append(move_row.get_text())
            for move_row in body_egg_moves.find_all("td",{"class":"cell-icon"}):
                ms_moveType_list.append((move_row.get_text()))
    
    #EGG MOVES
    table_counter = 0    
    tables = soup.find_all("table",{"class" : "data-table"})

    for table in tables:
        table_counter += 1
        
        if table_counter == 3:
            body_egg_moves = table.find("tbody")
            for move_row in body_egg_moves.find_all("td",{"class":"cell-name"}):
                ms_moves_list.append(move_row.get_text())
            for move_row in body_egg_moves.find_all("td",{"class":"cell-icon"}):
                ms_moveType_list.append((move_row.get_text()))
    
    #TM MOVES
    table_counter = 0    
    tables = soup.find_all("table",{"class" : "data-table"})

    for table in tables:
        table_counter += 1
        
        if table_counter == 4:
            body_tm_moves = table.find("tbody")
            for move_row in body_tm_moves.find_all("td",{"class":"cell-name"}):
                ms_moves_list.append(move_row.get_text())
            for move_row in body_tm_moves.find_all("td",{"class":"cell-icon"}):
                ms_moveType_list.append((move_row.get_text()))
    
    ms_moveType_list = ms_moveType_list[::2]
    ms_moves_dict = dict(zip(ms_moves_list,ms_moveType_list))
    
    return ms_moves_dict

## Choose best moveset

In [9]:
def MakeMoveset(pokemonID):        
    url = "https://gamepress.gg/pokemongo/pve-charge-moves"
    request = requests.get(url)
    soup = BeautifulSoup(request.text)

    pokemonType = scrapeDBType(pokemonID)
    type1 = pokemonType.get("Type1","")
    type2 = pokemonType.get("Type2","")

    ms_moves = scrapeDBMoves((pokemonID))

    highestc_Type1 = 0
    highestc_Type2 = 0
    totalHighestNamec = ""
    totalHighestTypec = ""
    moveType1 = ""
    moveType2 = ""
    try:
        moves_table = soup.find("table",{"id": "sort-table"})

        #CHARGED MOVE
        for table_row in moves_table.find_all("tr",class_=type1):
            for moveName in table_row.find_all("td",{"class":"views-field-field-energy-requirements-image"}):
                gamepress_move = moveName.get_text().strip()
                if gamepress_move in ms_moves.keys():
                    for moveDmg in table_row.find("td",{"class":"views-field-nothing"}):
                        if float(moveDmg) > highestc_Type1:
                            moveType1 = gamepress_move
                            highest_Type1 = float(moveDmg)    

        if type2 != "None":        
            for table_row in moves_table.find_all("tr",class_=type2):
                for moveName in table_row.find_all("td",{"class":"views-field-field-energy-requirements-image"}):
                    gamepress_move = moveName.get_text().strip()
                    if gamepress_move in ms_moves.keys():
                        for moveDmg in table_row.find("td",{"class":"views-field-nothing"}):
                            if float(moveDmg) > highestc_Type2:
                                moveType2 = gamepress_move
                                highest_Type2 = float(moveDmg)
        
        if type2 != "None":
            if highest_Type1 > highest_Type2:
                totalHighestNamec = moveType1
                totalHighestTypec = type1
            else:
                totalHighestNamec = moveType2
                totalHighestTypec = type2
        else:
            totalHighestNamec = moveType1
            totalHighestTypec = type1

        url = "https://gamepress.gg/pokemongo/pokemon-move/" + totalHighestNamec.lower().replace(" ","-")
        request = requests.get(url)
        soup = BeautifulSoup(request.text)

        moveTable = soup.find_all("table",{"class":"move-table"})  
        PvEstats = moveTable[1].find_all("div",{"class":"field field--name-field-move-dps field--type-computed-float field--label-hidden field__item"})
        DPSCharged = PvEstats[0].get_text()  

        #FAST MOVE
        url = "https://gamepress.gg/pokemongo/pve-fast-moves"
        request = requests.get(url)
        soup = BeautifulSoup(request.text)

        highestf_Type1 = 0
        highestf_Type2 = 0
        highestf_Total = 0
        totalHighestNamef = ""
        totalHighestTypef = ""
        moveType1 = ""
        moveType2 = ""

        moves_table = soup.find("table",{"id": "sort-table"})

        for table_row in moves_table.find_all("tr",class_=type1):
            for moveName in table_row.find_all("td",{"class":"views-field-field-energy-requirements-image"}):
                gamepress_move = moveName.get_text().strip()
                if gamepress_move in ms_moves.keys():
                    for moveDmg in table_row.find("td",{"class":"views-field-field-move-dps"}):
                        if float(moveDmg) > highestf_Type1:
                            moveType1 = gamepress_move
                            highest_Type1 = float(moveDmg)
        

        if type2 != "None":        
            for table_row in moves_table.find_all("tr",class_=type2):
                for moveName in table_row.find_all("td",{"class":"views-field-field-energy-requirements-image"}):
                    gamepress_move = moveName.get_text().strip()
                    if gamepress_move in ms_moves.keys():
                        for moveDmg in table_row.find("td",{"class":"views-field-field-move-dps"}):
                            if float(moveDmg) > highestf_Type2:
                                moveType2 = gamepress_move
                                highest_Type2 = float(moveDmg)
        
        if type2 != "None":
            if highest_Type1 > highest_Type2:
                totalHighestNamef = moveType1
                totalHighestTypef = type1
                highestf_Total = highest_Type1
            else:
                totalHighestNamef = moveType2
                totalHighestTypef = type2
                highestf_Total = highest_Type2
        else:
            totalHighestNamef = moveType1
            totalHighestTypef = type1
            highestf_Total = highest_Type1
        
        return{"Fast Attack" : totalHighestNamef,  "DPS Fast Attack" : highestf_Total,"Fast Type": totalHighestTypef, "Charged Attack": totalHighestNamec,"DPS Charged Attack": DPSCharged,"Charged Type": totalHighestTypec}
    except:
        return{"Fast Attack" : "None",  "DPS Fast Attack" : 0,"Fast Type": "None", "Charged Attack": "None","DPS Charged Attack": 0,"Charged Type": "None"}

In [10]:
MakeMoveset(384)

{'Fast Attack': 'Dragon Tail',
 'DPS Fast Attack': 13.64,
 'Fast Type': 'Dragon',
 'Charged Attack': 'Dragon Ascent',
 'DPS Charged Attack': '40.00',
 'Charged Type': 'Flying'}

### Converting stats

The scraped stats are not the final stats of a Pokemon. As pogo is a more simplified game, the stats for each pokemon are condensed to 3 instead of 6. 

Interessant: haal aan in het uitschrijven dat we perfecte IV's toevoegen aan elke stat

In [11]:
def statConversion(pokemonID):
    url = "https://pokemondb.net/pokedex/"+ str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)
    
    #POKEMON STATS
    table_counter = 0
    row_counter = 0
    
    pokemonName_stats = []

    all_stat_tables = soup.find_all("table",{"class": "vitals-table"})
    
    #print("\nPOKEMON MS STATS \n")

    for stat_table in all_stat_tables:
        table_counter += 1
        
        if table_counter == 4:
            
            for table_row in stat_table.find_all("tr"):                
                row_counter += 1
                
                if row_counter <= 6:
                    dimensions = table_row.find_all("td")
                    header = table_row.find("th")
                    #print(dimensions[0].get_text())
                    pokemonName_stats.append(dimensions[0].get_text())

    #print("\nPOKEMON GO STATS \n")

    #Speedmod
    speedMod = 1 + (int(pokemonName_stats[5])-75)/500

    #Attack
    scaledAttack = round(2*((7/8)*max(int(pokemonName_stats[1]),int(pokemonName_stats[3])) + (1/8)*min(int(pokemonName_stats[1]),float(pokemonName_stats[3]))))
    baseAttack = round(scaledAttack*speedMod) 
    baseAttackandIV = baseAttack + 15
    #print("Attack:",baseAttack)

    #Defence
    scaledDefense = round(2*((5/8)*max(int(pokemonName_stats[2]),int(pokemonName_stats[4]))+(3/8)*min(int(pokemonName_stats[2]),int(pokemonName_stats[4]))))
    baseDefense = round(scaledDefense*speedMod)
    baseDefenseandIV = baseDefense +15
    #print("Defense:",baseDefense)  

    #HP
    baseStamina = math.floor(int(pokemonName_stats[0]) * 1.75 + 50)
    baseStaminaandIV = baseStamina + 15
    #print("Stamina:",math.ceil(baseStamina),"\n") 

    #CP
    cplvl_40 = math.floor(max(10,(baseAttackandIV*(math.pow(baseDefenseandIV,0.5))*(math.pow(baseStaminaandIV,0.5))*(math.pow(0.7903,2)))/10))
    #print("before nerf:", cplvl_40)    
    if cplvl_40 >= 4000:
        baseStamina = math.ceil(baseStamina * 0.91) +15
        baseAttack = math.ceil(baseAttack * 0.91) +15
        baseDefense = round(baseDefense * 0.91) +15        
        cplvl_40 = round(max(10,((baseAttack)*(math.pow((baseDefense),0.5))*(math.pow((baseStamina),0.5))*(math.pow(0.7903,2)))/10))
        #print("after nerf:", cplvl_40)
        #print("\n",baseAttack - 15, baseDefense - 15, baseStamina - 15)
        return {"HP" : int(baseStamina)-15,"Attack": int(baseAttack),"Defense" : int(baseDefense)-15,"CP": int(cplvl_40)}
    else:
        return {"HP" : int(baseStamina),"Attack": int(baseAttack),"Defense" : int(baseDefense),"CP": int(cplvl_40)}
    
  

### PokemonGO moves

Hier gaan we te werk met data van https://gamepress.gg/pokemongo/ 

In [12]:
def gamepressScraperMoves1(pokemonID):
    url = "https://gamepress.gg/pokemongo/pokemon/"+str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)    
    fastAttack = "no fast attack"
    chargedAttack = "no charged attack"
    DPSFast = 0
    fastType = "None"
    DPSCharge = 0
    chargeType = "None"

    try:
        table = soup.find("table",{"class":"moveset-table"})
        tableBody = table.find("tbody")

        pvemoves = soup.find_all("h2",{"class": "main-title"})
        checkPVEmoves = True

        for title in pvemoves:
            
            if title.get_text().strip() != "PVE Movesets Grades":
                checkPVEmoves = False
            else:
                checkPVEmoves = True
                break

        if checkPVEmoves == False:
            return {"Fast Attack" : fastAttack,"DPS Fast Attack" : DPSFast,"Fast Type": fastType, "Charged Attack": chargedAttack,"DPS Charged Attack": DPSCharge,"Charged Type": chargeType}
        
            

        rows = tableBody.find_all("tr")

        if not rows:
            fastAttack = "no fast attack"
            chargedAttack = "no charged attack"
            return {"Fast Attack" : fastAttack,"DPS Fast Attack" : DPSFast,"Fast Type": fastType, "Charged Attack": chargedAttack,"DPS Charged Attack": DPSCharge,"Charged Type": chargeType}
        
        for row in rows:
            columns = row.find_all("td")

            if columns[2].get_text().strip() == "A":            
                fastAttack = columns[0].get_text().strip().replace('*','')
                chargedAttack = columns[1].get_text().strip().replace('*','')
                #return {"Fast Attack" : fastAttack, "Charged Attack": chargedAttack}

    except:
        fastAttack = "no fast attack"
        chargedAttack = "no charged attack"
        return {"Fast Attack" : fastAttack,"DPS Fast Attack" : DPSFast,"Fast Type": fastType, "Charged Attack": chargedAttack,"DPS Charged Attack": DPSCharge,"Charged Type": chargeType}
    
    ##MOVE METADATA
    
    fasturl = fastAttack.lower().replace('*','').replace(' ','-')
    chargedurl = chargedAttack.lower().replace('*','').replace(' ','-')

    fullfasturl = "https://gamepress.gg/pokemongo/pokemon-move/"+fasturl
    fullchargedurl = "https://gamepress.gg/pokemongo/pokemon-move/"+chargedurl

    requestfast = requests.get(fullfasturl)
    requestcharged = requests.get(fullchargedurl)

    ## FAST MOVE DATA

    try:
        fastSoup = BeautifulSoup(requestfast.text)
        fastTable = fastSoup.find_all("table",{"class":"move-table"})  
        fastType = fastTable[0].find_all("div",{"class":"field__item"})
        fastType = fastType[1].get_text()

        PvEstats = fastTable[1].find_all("div",{"class":"field field--name-field-move-dps field--type-computed-float field--label-hidden field__item"})
        DPSFast = PvEstats[0].get_text()
    except:
        return {"Fast Attack" : fastAttack,"DPS Fast Attack" : DPSFast,"Fast Type": fastType, "Charged Attack": chargedAttack,"DPS Charged Attack": DPSCharge,"Charged Type": chargeType}

    ## CHARGE MOVE TYPE
    try:
        chargeSoup = BeautifulSoup(requestcharged.text)
        chargeTable = chargeSoup.find_all("table",{"class":"move-table"})  
        chargeType = chargeTable[0].find_all("div",{"class":"field__item"})
        chargeType = chargeType[1].get_text()

        PvEstats = chargeTable[1].find_all("div",{"class":"field field--name-field-move-dps field--type-computed-float field--label-hidden field__item"})
        DPSCharge = PvEstats[0].get_text()
    except:
        return {"Fast Attack" : fastAttack,"DPS Fast Attack" : DPSFast,"Fast Type": fastType, "Charged Attack": chargedAttack,"DPS Charged Attack": DPSCharge,"Charged Type": chargeType}
    
    return {"Fast Attack" : fastAttack,  "DPS Fast Attack" : DPSFast,"Fast Type": fastType, "Charged Attack": chargedAttack,"DPS Charged Attack": DPSCharge,"Charged Type": chargeType}

### Pokemon Ranking

In [13]:
def gamepressScraperRanking(pokemonID):
    url = "https://gamepress.gg/pokemongo/pokemon/"+str(pokemonID)
    request = requests.get(url)
    soup = BeautifulSoup(request.text)

    try:
        all_ranks = ["S","A+","A","B+","B","C",]
        ranking = soup.find("span",{"class":"cell-rating-right"}).get_text()
        ranking = ranking.replace(" Tier","")        

        if ranking not in all_ranks:
            ranking = "F"
            return{"Rank": ranking}
        
        else:
            return{"Rank": ranking}

    except:
        ranking = "F"
        return{"Rank": ranking}

In [14]:
cols = ["PokemonName", "Type1","Type2","HP","Attack","Defense","CP","Fast Attack","DPS Fast Attack","Fast Type","Charged Attack","DPS Charged Attack","Charged Type","Rank"]
df = pd.DataFrame( columns=cols)
for i in range(1,3):
    data1 = scrapeDBName(i)
    data2 = scrapeDBType(i)
    data3 = statConversion(i)
    data4 = gamepressScraperMoves1(i)    
    time.sleep(1)
    df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))
df

<ipython-input-14-d2311b669aaa>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))
<ipython-input-14-d2311b669aaa>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([data1,data2,data3,data4], columns=cols))


,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,DPS Fast Attack,Fast Type,Charged Attack,DPS Charged Attack,Charged Type,Rank
0,Bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,128.0,118.0,111.0,1115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no fast attack,0.0,None,no charged attack,0.0,None,NaN
0,Ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,155.0,151.0,143.0,1699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no fast attack,0.0,None,no charged attack,0.0,None,NaN


In [15]:
df

,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,DPS Fast Attack,Fast Type,Charged Attack,DPS Charged Attack,Charged Type,Rank
0,Bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,128.0,118.0,111.0,1115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no fast attack,0.0,None,no charged attack,0.0,None,NaN
0,Ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Grass,Poison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,155.0,151.0,143.0,1699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no fast attack,0.0,None,no charged attack,0.0,None,NaN


In [16]:
notIncluded = []        
pokemon_availability = {}

i = 0
url = "https://serebii.net/pokemongo/gen"+str(1)+"pokemon.shtml"
request = requests.get(url)
print(request.status_code)

while request.status_code == 200:
    i += 1
    print(i, request.status_code)
        
    url = "https://serebii.net/pokemongo/gen"+str(i)+"pokemon.shtml"
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')

    if request.status_code == 200:
        table = soup.find_all("table", {"class": "tab"})
        tableBody = table[1]


        for row in tableBody.find_all("tr"):
            dimension = row.find_all("td", {"class": "fooinfo"})

            if len(dimension) == 3:
                name = dimension[0].get_text()
                number = row.find("td").get_text().strip()
                number = int(number.replace('#', ''))

                if "Not Currently Available" in name or number in notIncluded:
                    pokemon_availability[number] = False
                    notIncluded.append(number)

                else:
                    pokemon_availability[number] = True

            available_pokemon_ids = [pokemon_id for pokemon_id, is_available in pokemon_availability.items() if is_available]
        
    else:
        break    

available_pokemon_ids.sort()

for item in notIncluded:
    print(item)

200
1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
489
490
493
679
680
681
681
719
719
721
746
746
749
750
771
772
773
774
774
774
774
774
774
774
774
778
780
781
800
800
800
800
801
801
802
803
804
805
806
807
810
811
812
813
814
815
816
817
818
821
822
823
824
825
826
827
828
829
830
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
849
850
851
852
853
854
855
856
857
858
859
860
861
864
868
869
871
872
873
874
875
875
876
876
877
877
878
879
880
881
882
883
884
885
886
887
890
890
891
892
892
896
897
898
898
898
917
918
924
925
925
926
927
931
932
933
934
935
936
937
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
963
964
964
965
966
967
968
969
970
973
976
977
978
978
978
978
979
981
982
982
982
983
984
985
986
987
988
989
990
991
992
993
994
995
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1024
1024


## Get all existing Pokémon

In [17]:
pokemon_data_list = []

for i in range(0,len(available_pokemon_ids)):
    data1 = scrapeDBName(available_pokemon_ids[i])
    data2 = scrapeDBType(available_pokemon_ids[i])
    data3 = statConversion(available_pokemon_ids[i])
    data4 = gamepressScraperMoves1(available_pokemon_ids[i])
    data5 = gamepressScraperRanking(available_pokemon_ids[i])
    time.sleep(1)
    
    # Merge alle dictionaries 
    merged_data = {**data1, **data2, **data3, **data4, **data5}    
    pokemon_data_list.append(merged_data)

df_existing = pd.DataFrame(pokemon_data_list)

In [18]:
df_existing

,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,DPS Fast Attack,Fast Type,Charged Attack,DPS Charged Attack,Charged Type,Rank
0,Bulbasaur,Grass,Poison,128,118,111,1115,no fast attack,0,None,no charged attack,0,None,F
1,Ivysaur,Grass,Poison,155,151,143,1699,no fast attack,0,None,no charged attack,0,None,F
2,Venusaur,Grass,Poison,190,198,189,2720,Vine Whip,11.67,Grass,Frenzy Plant,38.46,Grass,B
3,Charmander,Fire,None,118,116,93,980,no fast attack,0,None,no charged attack,0,None,F
4,Charmeleon,Fire,None,151,158,126,1653,no fast attack,0,None,no charged attack,0,None,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,Frigibax,Dragon,Ice,163,134,86,1247,no fast attack,0,None,no charged attack,0,None,F
824,Arctibax,Dragon,Ice,207,173,128,2092,no fast attack,0,None,no charged attack,0,None,F
825,Baxcalibur,Dragon,Ice,229,269,167,3541,Ice Fang,8.00,Ice,Avalanche,33.33,Ice,B+
826,Gimmighoul,Ghost,None,128,121,122,1188,no fast attack,0,None,no charged attack,0,None,F


In [19]:
df_existing.to_csv("pokemon_dataset_available.csv")

## Get unavailable Pokémon

In [20]:
pokemon_data_list = []

for i in range(0,len(notIncluded)):
    data1 = scrapeDBName(notIncluded[i])
    data2 = scrapeDBType(notIncluded[i])
    data3 = statConversion(notIncluded[i])
    data4 = MakeMoveset(notIncluded[i])
    time.sleep(1)
    
    # Merge alle dictionaries 
    merged_data = {**data1, **data2, **data3, **data4}    
    pokemon_data_list.append(merged_data)

df_unreleased = pd.DataFrame(pokemon_data_list)

In [21]:
df_unreleased

,PokemonName,Type1,Type2,HP,Attack,Defense,CP,Fast Attack,DPS Fast Attack,Fast Type,Charged Attack,DPS Charged Attack,Charged Type
0,Phione,Water,None,190,162,162,2105,Waterfall,13.33,Water,Liquidation,23.33,Water
1,Manaphy,Water,None,225,210,210,3265,Waterfall,13.33,Water,Liquidation,23.33,Water
2,Arceus,Normal,None,237,254,238,4006,Take Down,6.67,Normal,Giga Impact,42.55,Normal
3,Honedge,Steel,Ghost,128,135,139,1390,,38.78,Steel,Gyro Ball,24.24,Steel
4,Doublade,Steel,Ghost,153,188,206,2443,,38.78,Steel,Gyro Ball,24.24,Steel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Iron Boulder,Rock,Psychic,207,249,214,3717,Rock Throw,13.33,Rock,Meteor Beam,73.68,Rock
214,Iron Crown,Steel,Psychic,207,243,220,3680,Metal Claw,11.43,Steel,Future Sight,44.44,Psychic
215,Terapagos,Normal,None,207,126,165,1760,Take Down,6.67,Normal,Weather Ball,34.38,Normal
216,Terapagos,Normal,None,207,126,165,1760,Take Down,6.67,Normal,Weather Ball,34.38,Normal


In [22]:
df_unreleased.to_csv("pokemon_dataset_unavailable.csv")

## Uploading files to DB

In [29]:
mydb = mysql.connector.connect(
    host = "sql11.freesqldatabase.com",
    user = "sql11676692",
    password = "NaNRRSxBTq",
    database = "sql11676692"
)
 
cursor = mydb.cursor()
 
load_data_query = f"""
    LOAD DATA INFILE 'pokemon_dataset_unavailable.csv'
    INTO TABLE unreleased
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 0 ROWS;
"""

cursor.execute(load_data_query)
mydb.commit()
cursor.close()
mydb.close()

ProgrammingError: 1045 (28000): Access denied for user 'sql11676692'@'%' (using password: YES)

In [ ]:
mydb = mysql.connector.connect(
    host = "sql11.freesqldatabase.com",
    user = "sql11676692",
    password = "NaNRRSxBTq",
    database = "sql11676692"
)
 
cursor = mydb.cursor()
 
load_data_query = f"""
    LOAD DATA INFILE 'pokemon_dataset_available.csv'
    INTO TABLE existing
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\\n'
    IGNORE 0 ROWS;
"""

cursor.execute(load_data_query)
mydb.commit()
cursor.close()
mydb.close()